In [ ]:
!pip install -q -U albumentations
!pip install roboflow
%pip uninstall opencv-python-headless
%pip install opencv-python-headless==4.1.2.30

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from matplotlib import pyplot as plt
from roboflow import Roboflow


DIR_INPUT = 'YOUR_DATASET_DIR'
DIR_TRAIN = f'{DIR_INPUT}/train'
DIR_TEST = f'{DIR_INPUT}/test'

In [ ]:
#  Downoad dataset from Roboflow in TF object detection format (CSV)

rf = Roboflow(api_key="YOUR API_KEY")
project = rf.workspace("WORKSPACE").project("PROJECT")
dataset = project.version(4).download("tensorflow")

In [ ]:
# Rename the images to remove invalid characters that are added by roboflow

for file in list(os.walk(DIR_TRAIN))[0][2]:
  if file[-3:] == "jpg":
    new_file_name = file[0:11] + file[-8:-5] + ".jpg"
    os.rename(f"{DIR_TRAIN}/{file}", f"{DIR_TRAIN}/{new_file_name}")

In [ ]:
train_df = pd.read_csv(f'{DIR_INPUT}/train/_annotations.csv')

# Beautify annotation
train_df['image_id'] = train_df['filename'].str.slice(0,11) + train_df['filename'].str.slice(-8,-5)
train_df['x'] = train_df['xmin'].astype(np.float64)
train_df['y'] = train_df['ymin'].astype(np.float64)
train_df['w'] = (train_df['xmax'] - train_df['xmin']).astype(np.float64)
train_df['h'] = (train_df['ymax'] - train_df['ymin']).astype(np.float64)
train_df['source'] = train_df['class'].astype(np.int64)
train_df.drop(columns=['width', 'height', 'xmin', 'xmax', 'ymin', 'ymax', 'filename', 'class'], inplace=True)

# Split
image_ids = train_df['image_id'].unique()
valid_ids = image_ids[394:]
train_ids = image_ids[:394]

In [ ]:
class WheatDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()
        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]
        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_dataset = WheatDataset(train_df, DIR_TRAIN, get_train_transform())
valid_dataset = WheatDataset(valid_df, DIR_TRAIN, get_valid_transform())
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=5,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=5,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

In [ ]:
def get_train_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

In [ ]:
loss_hist = Averager()
itr = 1
num_epochs = 50

for epoch in range(num_epochs):
    loss_hist.reset()
    for images, targets, image_ids in train_data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        loss_hist.send(loss_value)
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")
        itr += 1
    print(f"Epoch #{epoch} loss: {loss_hist.value}")   

In [ ]:
torch.save(model.state_dict(), 'fasterrcnn_resnet50_fpn.pth')